In [ ]:
# 라이브러리 import
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib
import warnings
from xgboost import XGBClassifier

In [ ]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터 로드
URL = "/content/drive/MyDrive/SIL/WORK/DSS 머신러닝2/" # 데이터 위치 설정
data = pd.read_csv(URL+"data/class_balance.csv",encoding="EUC-KR")

In [ ]:
data['Y'] = data['Y'].map(lambda x: 0 if x == -1 else 1) # 기존 -1, 1이었던 Y값을 모델 학습을 위해 0, 1로 변경

In [ ]:
# X, Y 분할
Y=data["Y"].copy()
X=data.drop("Y",axis=1)
X.head(3)

# 데이터셋을 학습 세트와 테스트 세트로 분리
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=22,shuffle =True)

# 특성 스케일링 (선택 사항)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# 평가 지표 출력 함수
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix

def print_all_reg(Y_test,pred):
    # Specificity를 구하기 위해 confusion matrix를 이용
    cm1 = confusion_matrix(Y_test,pred)
    specificity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])

    #결과 검사
    #recall = cm1[1,1]/(cm1[1,1]+cm1[1,0])
    #pre = cm1[1,1]/(cm1[1,1]+cm1[0,1])

    G_mean = recall_score(Y_test,pred) * specificity1

    print("model의 recall 값은 {:.3f}".format(recall_score(Y_test,pred)))
    print("model의 2종 오류 확률 값은 {:.3f}".format(1-recall_score(Y_test,pred)))
    print("model의 Specificity 값은 {:.3f}".format(specificity1))
    print("model의 1종 오류 확률 값은 {:.3f}".format(1-specificity1))
    print("model의 precision 값은 {:.3f}".format(precision_score(Y_test,pred)))
    print("model의 f1_score 값은 {:.3f}".format(f1_score(Y_test,pred)))
    print("model의 G-mean 값은 {:.3f}".format(np.sqrt(G_mean)))
    print("model의 accuracy 값은 {:.3f}".format(accuracy_score(Y_test,pred)))

[[StackingClassifier Parameters](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html)]
### **Parameters**
- **estimators** :기본 모델들의 리스트. 각 튜플은 (이름, 모델)의 형태
- **final_estimator** : 메타 모델, 기본 모델들의 예측 결과를 입력으로 받아 최종 예측을 수행
- **cv** : 교차 검증 분할 수, 기본 모델의 학습을 위한 교차 검증 방법을 설정
- **stack_method** : 기본 모델의 예측 결과를 사용할 방법, 'auto', 'predict_proba', 'decision_function', 'predict' 중 하나
  - 'auto': 분류와 회귀 문제에 따라 자동으로 예측 방법을 선택함
  - 'predict_proba': 분류 문제에서 사용됨. predict_proba는 각 클래스에 대한 확률을 반환하므로, 메타 모델은 이러한 확률 정보를 입력으로 사용하여 최종 예측을 수행함
   - 제약 조건: 모든 기본 모델이 decision_function 메서드를 지원해야 함(대부분 지원)

  - 'decision_function': 분류 문제에서 사용함. decision_function은 각 클래스에 대한 점수를 반환하며, 모델이 학습한 선형 방정식의 출력을 반환함
    - 제약 조건: 모든 기본 모델이 decision_function 메서드를 지원해야 함(주로 선형 모델이나 SVM과 같은 일부 모델에서 지원함)

  - 'predict':분류 및 회귀 문제에서 모두 사용함. predict 메서드는 분류 문제에서는 클래스 레이블, 회귀 문제에서는 예측 값을 반환함.
    - 제약 조건: 모든 기본 모델이 predict 메서드를 지원해야 합니다 (기본적으로 모든 모델이 predict를 지원함).
- **n_jobs** :  병렬 처리를 위한 CPU 코어 수, None이면 1, -1이면 모든 CPU 코어 사용
- **passthrough** : True이면 원본 특성을 메타 모델에 추가로 전달, False이면 기본 모델의 예측 결과만 메타 모델에 전달

decision_function vs predict_proba


In [ ]:
# 기본 모델 정의
level0 = [
    ('lr', LogisticRegression(max_iter=1000, C=1.0, solver='lbfgs')),
    ('svm', SVC(probability=True, C=1.0, kernel='rbf', gamma='scale')),
    ('rf', RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=1)),
    ('xgb', XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss'))
]

# 개별 모델 학습 및 성능 평가
for name, model in level0:
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    print(f'\n{name} Model Performance:')
    print_all_reg(Y_test, Y_pred)

# 메타 모델 정의
level1 = LogisticRegression()

# 스태킹 모델 정의
stacking_model = StackingClassifier(
    estimators=level0,  # 기본 모델들의 리스트. 각 튜플은 (이름, 모델)의 형태
    final_estimator=level1,  # 메타 모델, 기본 모델들의 예측 결과를 입력으로 받아 최종 예측을 수행
    cv=5,  # 교차 검증 분할 수, 기본 모델의 학습을 위한 교차 검증 방법을 설정
    stack_method='auto',  # 기본 모델의 예측 결과를 사용할 방법, 'auto', 'predict_proba', 'decision_function', 'predict' 중 하나
    n_jobs=None,  # 병렬 처리를 위한 CPU 코어 수, None이면 1, -1이면 모든 CPU 코어 사용
    passthrough=False  # True이면 원본 특성을 메타 모델에 추가로 전달, False이면 기본 모델의 예측 결과만 메타 모델에 전달
)
# 스태킹 모델 학습
stacking_model.fit(X_train, Y_train)


lr Model Performance:
model의 recall 값은 0.778
model의 2종 오류 확률 값은 0.222
model의 Specificity 값은 0.848
model의 1종 오류 확률 값은 0.152
model의 precision 값은 0.737
model의 f1_score 값은 0.757
model의 G-mean 값은 0.812
model의 accuracy 값은 0.824

svm Model Performance:
model의 recall 값은 0.667
model의 2종 오류 확률 값은 0.333
model의 Specificity 값은 0.939
model의 1종 오류 확률 값은 0.061
model의 precision 값은 0.857
model의 f1_score 값은 0.750
model의 G-mean 값은 0.791
model의 accuracy 값은 0.843

rf Model Performance:
model의 recall 값은 0.667
model의 2종 오류 확률 값은 0.333
model의 Specificity 값은 1.000
model의 1종 오류 확률 값은 0.000
model의 precision 값은 1.000
model의 f1_score 값은 0.800
model의 G-mean 값은 0.816
model의 accuracy 값은 0.882

xgb Model Performance:
model의 recall 값은 0.722
model의 2종 오류 확률 값은 0.278
model의 Specificity 값은 0.939
model의 1종 오류 확률 값은 0.061
model의 precision 값은 0.867
model의 f1_score 값은 0.788
model의 G-mean 값은 0.824
model의 accuracy 값은 0.863


StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(max_iter=1000)),
                               ('svm', SVC(probability=True)),
                               ('rf', RandomForestClassifier(max_depth=10)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eva...
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...))],
                   final_estimator=LogisticRegression())

In [ ]:
# 스태킹 모델 예측 수행
Y_pred = stacking_model.predict(X_test)

# 스태킹 모델 성능 평가
print(f'\nStacking Model Performance:')
print_all_reg(Y_test, Y_pred)


Stacking Model Performance:
model의 recall 값은 0.833
model의 2종 오류 확률 값은 0.167
model의 Specificity 값은 0.909
model의 1종 오류 확률 값은 0.091
model의 precision 값은 0.833
model의 f1_score 값은 0.833
model의 G-mean 값은 0.870
model의 accuracy 값은 0.882
